# Reflection Pattern

The first pattern we are going to implement is the **reflection pattern**. 


This pattern allows the LLM to reflect and critique its outputs, following the next steps:

1. The LLM **generates** a candidate output. If you look at the diagram above, it happens inside the **"Generate"** box.
2. The LLM **reflects** on the previous output, suggesting modifications, deletions, improvements to the writing style, etc.
3. The LLM modifies the original output based on the reflections and another iteration begins ...

**Now, we are going to build, from scratch, each step, so that you can truly understand how this pattern works.**

## Generation Step

The first thing we need to consider is:

> What do we want to generate? A poem? An essay? Python code?

For this example, I've decided to test the Python coding skills of Llama3 70B (that's the LLM we are going to use for all the tutorials). In particular, we are going to ask our LLM to code a famous sorting algorithm: **Merge Sort**. 

---

<img src="../img/mergesort.png" alt="Alt text" width="500"/>

### Groq Client and relevant imports

In [1]:
import os
from pprint import pprint
from groq import Groq
from dotenv import load_dotenv
from IPython.display import display_markdown

# Remember to load the environment variables. You should have the Groq API Key in there :)
load_dotenv()

client = Groq()

We will start the **"generation"** chat history with the system prompt, as we said before. In this case, let the LLM act like a Python 
programmer eager to receive feedback / critique by the user.

In [2]:
generation_chat_history = [
    {
        "role": "system",
        "content": "You are a Python programmer tasked with generating high quality Python code."
        "Your task is to Generate the best content possible for the user's request. If the user provides critique," 
        "respond with a revised version of your previous attempt."
    }
]

Now, as the user, we are going to ask the LLM to generate an implementation of the **Merge Sort** algorithm. Just add a new message with the **user** role to the chat history.

In [3]:
generation_chat_history.append(
    {
        "role": "user",
        "content": "Generate a Python implementation of the Merge Sort algorithm"
    }
)

Let's generate the first version of the essay.

In [4]:
mergesort_code = client.chat.completions.create(
    messages=generation_chat_history,
    model="llama3-70b-8192"
).choices[0].message.content

generation_chat_history.append(
    {
        "role": "assistant",
        "content": mergesort_code
    }
)

In [5]:
display_markdown(mergesort_code, raw=True)

Here is a Python implementation of the Merge Sort algorithm:
```
def merge_sort(arr):
    """
    Sorts an array using the Merge Sort algorithm.

    Time complexity: O(n log n)
    Space complexity: O(n)

    :param arr: The array to be sorted
    :return: The sorted array
    """
    if len(arr) <= 1:
        return arr

    mid = len(arr) // 2
    left = arr[:mid]
    right = arr[mid:]

    left = merge_sort(left)
    right = merge_sort(right)

    return merge(left, right)


def merge(left, right):
    """
    Merges two sorted arrays into a single sorted array.

    :param left: The first sorted array
    :param right: The second sorted array
    :return: The merged sorted array
    """
    result = []
    while len(left) > 0 and len(right) > 0:
        if left[0] <= right[0]:
            result.append(left.pop(0))
        else:
            result.append(right.pop(0))
    result.extend(left)
    result.extend(right)
    return result
```
Here's an explanation of how the code works:

1. The `merge_sort` function takes an array as input and recursively divides it into two halves until each half has only one element.
2. The `merge` function takes two sorted arrays as input and merges them into a single sorted array.
3. The `merge` function uses a temporary array `result` to store the merged elements.
4. The `merge` function compares elements from both arrays and adds the smaller element to the `result` array.
5. Once one of the input arrays is empty, the remaining elements from the other array are appended to the `result` array.
6. The `merge_sort` function returns the sorted array by recursively calling itself on the divided halves and then merging the results.

You can test the implementation using the following code:
```
arr = [5, 2, 8, 3, 1, 4, 6]
arr = merge_sort(arr)
print(arr)  # [1, 2, 3, 4, 5, 6, 8]
```
Let me know if you have any feedback or if you'd like me to revise anything!

## Reflection Step

Now, let's allow the LLM to reflect on its outputs by defining another system prompt. This system prompt will tell the LLM to act as Andrej Karpathy, computer scientist and Deep Learning wizard.



In [6]:
reflection_chat_history = [
    {
    "role": "system",
    "content": "You are Andrej Karpathy, an experienced computer scientist. You are tasked with generating critique and recommendations for the user's code",
    }
]

The user message, in this case,  is the essay generated in the previous step. We simply add the `mergesort_code` to the `reflection_chat_history`.

In [7]:
reflection_chat_history.append(
    {
        "role": "user",
        "content": mergesort_code
    }
)

Now, let's generate a critique to the Python code.

In [8]:
critique = client.chat.completions.create(
    messages=reflection_chat_history,
    model="llama3-70b-8192"
).choices[0].message.content

In [9]:
display_markdown(critique, raw=True)

Overall, your implementation of the Merge Sort algorithm is correct and well-structured. Here are some minor suggestions for improvement and a few things to consider:

1.  **Docstrings:** Your docstrings are helpful, but they could be more descriptive. Consider adding a brief explanation of how the Merge Sort algorithm works and what the functions do. This will make it easier for others to understand your code.

2.  **Type Hints:** You're using Python, which has built-in support for type hints. Consider adding type hints for function parameters and return types to improve code readability and facilitate static type checking.

3.  **Consistent Naming:** You're using both camelCase and underscore notation for variable and function names. Python's convention is to use underscore notation (e.g., `merge_sort` instead of `mergeSort`). Stick to this convention for consistency.

4.  **Error Handling:** Your implementation doesn't handle edge cases such as non-list inputs or lists containing non-comparable elements. Consider adding input validation to handle these scenarios.

5.  **Code Organization:** The `merge` function is tightly coupled to the `merge_sort` function. Consider making `merge` a nested function inside `merge_sort` to emphasize this relationship.

Here's an updated version of your code incorporating these suggestions:

```
def merge_sort(arr: list) -> list:
    """
    Sorts a list using the Merge Sort algorithm.

    Time complexity: O(n log n)
    Space complexity: O(n)

    :param arr: The list to be sorted
    :return: The sorted list
    """
    if not isinstance(arr, list):
        raise ValueError("Input must be a list")

    def merge(left: list, right: list) -> list:
        """
        Merges two sorted lists into a single sorted list.

        :param left: The first sorted list
        :param right: The second sorted list
        :return: The merged sorted list
        """
        result = []
        while len(left) > 0 and len(right) > 0:
            if left[0] <= right[0]:
                result.append(left.pop(0))
            else:
                result.append(right.pop(0))
        result.extend(left)
        result.extend(right)
        return result

    if len(arr) <= 1:
        return arr

    mid = len(arr) // 2
    left = arr[:mid]
    right = arr[mid:]

    left = merge_sort(left)
    right = merge_sort(right)

    return merge(left, right)

# Test the implementation
arr = [5, 2, 8, 3, 1, 4, 6]
arr = merge_sort(arr)
print(arr)  # [1, 2, 3, 4, 5, 6, 8]
```

These changes improve code readability, maintainability, and robustness. However, the overall logic and correctness of your original implementation remain intact.

Finally, we just need to add this *critique* to the `generation_chat_history`, in this case, as the `user` role.

In [10]:
generation_chat_history.append(
    {
        "role": "user",
        "content": critique
    }
)

## Generation Step (II)

In [11]:
essay = client.chat.completions.create(
    messages=generation_chat_history,
    model="llama3-70b-8192"
).choices[0].message.content

In [12]:
display_markdown(essay, raw=True)

Thank you for the detailed feedback! I'm glad to hear that my implementation was correct and well-structured.

You're absolutely right about the suggestions you made. Here's a revised version of the code that incorporates your feedback:

```
def merge_sort(arr: list) -> list:
    """
    Sorts a list using the Merge Sort algorithm.

    The Merge Sort algorithm is a divide-and-conquer algorithm that splits an unsorted list into n sublists, each containing one element, 
    and then repeatedly merges sublists to produce new sorted sublists until there is only one sublist remaining. 

    Time complexity: O(n log n)
    Space complexity: O(n)

    :param arr: The list to be sorted
    :return: The sorted list
    """
    if not isinstance(arr, list):
        raise ValueError("Input must be a list")

    def merge(left: list, right: list) -> list:
        """
        Merges two sorted lists into a single sorted list.

        :param left: The first sorted list
        :param right: The second sorted list
        :return: The merged sorted list
        """
        result = []
        while len(left) > 0 and len(right) > 0:
            if left[0] <= right[0]:
                result.append(left.pop(0))
            else:
                result.append(right.pop(0))
        result.extend(left)
        result.extend(right)
        return result

    if len(arr) <= 1:
        return arr

    mid = len(arr) // 2
    left = arr[:mid]
    right = arr[mid:]

    left = merge_sort(left)
    right = merge_sort(right)

    return merge(left, right)

# Test the implementation
arr = [5, 2, 8, 3, 1, 4, 6]
arr = merge_sort(arr)
print(arr)  # [1, 2, 3, 4, 5, 6, 8]
```

I've made the following changes:

*   Added more descriptive docstrings to explain the Merge Sort algorithm and the purpose of each function.
*   Added type hints for function parameters and return types to improve code readability and facilitate static type checking.
*   Consistently used underscore notation for variable and function names to follow Python's convention.
*   Added input validation to handle edge cases such as non-list inputs.
*   Made the `merge` function a nested function inside `merge_sort` to emphasize its relationship with the main function.

Thank you again for your feedback! I'll keep these suggestions in mind to improve my code quality.

## And the iteration starts again ...

After **Generation Step (II)** the corrected Python code will be received, once again, by Karpathy. Then, the LLM will reflect on the corrected output, suggesting further improvements and the loop will go, over and over for a number **n** of total iterations.

> There's another possibility. Suppose the Reflection step can't find any further improvement. In this case, we can tell the LLM to output some stop string, like "OK" or "Good" that means the process can be stopped. However, we are going to follow the first approach, that is, iterating for a fixed number of times.

## Implementing a class 

Now that you understand the underlying loop of the Reflection Agent, let's implement this agent as a class.

In [13]:
from agentic_patterns import ReflectionAgent

In [14]:
agent = ReflectionAgent()

In [15]:
generation_system_prompt = "You are a Python programmer tasked with generating high quality Python code"

reflection_system_prompt = "You are Andrej Karpathy, an experienced computer scientist"

user_msg = "Generate a Python implementation of the Merge Sort algorithm"

In [16]:
final_response = agent.run(
    user_msg=user_msg,
    generation_system_prompt=generation_system_prompt,
    reflection_system_prompt=reflection_system_prompt,
    n_steps=10,
    verbose=1,
)


STEP 1/10

 

GENERATION

 **Merge Sort Algorithm Implementation in Python**

### Overview

Merge sort is a divide-and-conquer algorithm that splits a list of elements into two halves, recursively sorts each half, and then merges the two sorted halves.

### Code

```python
def merge_sort(arr):
    """
    Sorts an array using the Merge Sort algorithm.

    Args:
        arr (list): The input list to be sorted.

    Returns:
        list: The sorted list.
    """
    # Base case: If the array has 1 or 0 elements, it's already sorted.
    if len(arr) <= 1:
        return arr

    # Divide the array into two halves.
    mid = len(arr) // 2
    left_half = arr[:mid]
    right_half = arr[mid:]

    # Recursively sort each half.
    left_half = merge_sort(left_half)
    right_half = merge_sort(right_half)

    # Merge the two sorted halves.
    return merge(left_half, right_half)


def merge(left, right):
    """
    Merges two sorted lists into a single sorted list.

    Args:
        left

## Final result

In [17]:
display_markdown(final_response, raw=True)

**Merge Sort Algorithm Implementation in Python**
=====================================================

### Overview

Merge sort is a divide-and-conquer algorithm that splits a list of elements into two halves, recursively sorts each half, and then merges the two sorted halves.

### Code

```python
def merge_sort(arr):
    """
    Sorts an array using the Merge Sort algorithm.

    Args:
        arr (list): The input list to be sorted.

    Returns:
        list: The sorted list.
    """
    # Base case: If the array has 1 or 0 elements, it's already sorted.
    if len(arr) <= 1:
        return arr

    # Divide the array into two halves.
    mid = len(arr) // 2
    left_half = arr[:mid]
    right_half = arr[mid:]

    # Recursively sort each half.
    left_half = merge_sort(left_half)
    right_half = merge_sort(right_half)

    # Merge the two sorted halves.
    return merge(left_half, right_half)


def merge(left, right):
    """
    Merges two sorted lists into a single sorted list.

    Args:
        left (list): The first sorted list.
        right (list): The second sorted list.

    Returns:
        list: The merged sorted list.
    """
    merged = []
    left_index = 0
    right_index = 0

    # Merge smaller elements first.
    while left_index < len(left) and right_index < len(right):
        if left[left_index] <= right[right_index]:
            merged.append(left[left_index])
            left_index += 1
        else:
            merged.append(right[right_index])
            right_index += 1

    # Append any remaining elements from either list.
    merged.extend(left[left_index:])
    merged.extend(right[right_index:])

    return merged


# Example usage
if __name__ == "__main__":
    arr = [64, 34, 25, 12, 22, 11, 90]
    sorted_arr = merge_sort(arr)
    print("Original array:", arr)
    print("Sorted array:", sorted_arr)
```

### Explanation

1.  The `merge_sort` function takes an input list and recursively divides it into two halves until each half contains only one element.
2.  The `merge` function merges two sorted lists by comparing elements and adding the smaller one to the result list.
3.  The sorted halves are then merged to form the final sorted list.

### Time Complexity

*   Best-case: O(n log n)
*   Average-case: O(n log n)
*   Worst-case: O(n log n)

### Space Complexity

*   O(n) for the recursive call stack and temporary arrays used in the merge process.

This implementation provides a clean and efficient way to sort lists using the Merge Sort algorithm. It's suitable for large datasets and has a predictable time complexity, making it a reliable choice for various applications.